<a href="https://colab.research.google.com/github/damithdev/AnimTrap/blob/master/Twitter_Scraping_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Text Analytics Coursework - MSc in Big Data Analytics
#### July 2020

###### Name: Damith Warnakulasuriya
###### IIT ID: 20191273
###### RGU ID: 1915109

### Answer (a)

Initializing necessary configurations

In [ ]:
!pip install twitter # Installig Twitter


     |████████████████████████████████| 61kB 1.8MB/s 


###Constants

In [ ]:
CONSUMER_KEY = 'C1El96xn4a6MF7WiStKCtWddQ'
CONSUMER_SECRET = '58F2JF3POHCXv29pm7mZUCoo7NKpN4F5NwV2KZv0AUqXO0i5Ua'
OAUTH_TOKEN = '382034120-sH3bbj7RlOK7Lff1z8gvdjbL1E2QXRBUB4WXyel0'
OAUTH_TOKEN_SECRET = 'K6tNUIhSqCjgBg3Xg7wLD7XiQGsr861I2MSeRIwBVIx22'

bbc_screen_name = "BBCWorld"
cnn_screen_name = "CNN"

screen_names = [cnn_screen_name,bbc_screen_name]
min_pull_count = 10000;
item_pull_count = round(min_pull_count/len(screen_names))



Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Initializing Twitter API

In [ ]:
import twitter # Twitter lib


auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth) # Initialize twitter api

type(twitter_api)

twitter.api.Twitter

Necessary Imports

In [2]:
import traceback
from urllib.parse import urlparse # parse urls into 6 components
from bs4 import BeautifulSoup # for pulling data out of HTML and XML files
import re
import requests
import time
import csv
import pandas as pd 

**function get_end_url**

*get expanded url from the tweet object*
*if the url is a tweet url get the tweet html and parse the website url*

returns re-directed webpage url

In [ ]:


def get_end_url(item):
  # print(item)
  try:
    url = item['entities']['urls'][0]['expanded_url']
    r = requests.get(url)
    # print(r.url)
    parsed_uri = urlparse(r.url)
    result = '{uri.netloc}'.format(uri=parsed_uri)
    # print(result)
    if 'twitter' in result:
      id_of_tweet = item['id']
      time.sleep(4)
      html = twitter_api.statuses.oembed(_id=id_of_tweet)['html']
      # print(html)
      soup = BeautifulSoup(html)
      url = soup.p.a['href']
      r = requests.get(url)

    url = r.url

    # print(str(itr) +":"+ url)

    parsed_uri = urlparse(r.url)
    result = '{uri.netloc}'.format(uri=parsed_uri)
    if 'twitter' in result:
      return None;
    return url
  
  except Exception as e:
    print("exception")
    print(e)
    traceback.print_exc()
    return None    


builds the urls list until it reaches the minimum url count

fetch twitter user timeline continuously backwards,
extract url then find the end url using above method

then backup the urls to google drive for future use

In [ ]:


tweets = []
end_urls = []
newslist = {}

itr = 0

for item in screen_names:
  print(item)
  
  retries = 0;
  item_tweets = []
  max_id = None
  item_urls = []

  while len(item_urls) < item_pull_count:
    if retries > 30 : break;

    if max_id is not None:
      res = twitter_api.statuses.user_timeline(screen_name=item,count=item_pull_count,max_id=max_id)
    else :
      res = twitter_api.statuses.user_timeline(screen_name=item,count=item_pull_count)
    item_tweets += res
    last = res[len(res)-1]
    max_id = last['id']
    print(len(item_tweets))
    retries =+ 1

    for tweet in res:
      # if itr > 5 :
      #   retries = 11;
      #   itr = 0
      #   break;

      urls = tweet['entities']['urls']
      if urls == [] : continue
      url = urls[0]['expanded_url']
      newslist[url] = tweet
      resurl = get_end_url(tweet)
      if resurl is not None : 
        item_urls.append(resurl)
        itr += 1
        print(str(itr)+" - " + resurl)

  tweets += item_tweets
  end_urls += item_urls

print("TOTAL TWEETS:" + str(len(tweets)))
print("TOTAL URLS:" + str(len(end_urls)))


#Save to csv using panas for reference
tempdf = pd.DataFrame(end_urls)
csv_data = tempdf.to_csv('news_urls_big.csv',index=False)
!cp news_urls_big.csv "/content/gdrive/My Drive/colab_data"

print("COMPLETE")

CNN
200
1 - https://www.cnn.com/2020/07/21/business/walmart-thanksgiving-day-closed/index.html?utm_term=link&utm_source=twCNN&utm_medium=social&utm_content=2020-07-21T17%3A53%3A03
2 - https://www.cnn.com/2020/07/21/asia/india-yellow-turtle-found-scli-intl/index.html?utm_source=twCNN&utm_medium=social&utm_content=2020-07-21T17%3A44%3A04&utm_term=link
3 - https://www.cnn.com/2020/07/21/health/cinnamon-prediabetes-wellness/index.html?utm_source=twCNN&utm_medium=social&utm_content=2020-07-21T17%3A35%3A01&utm_term=link
4 - https://www.cnn.com/2020/07/21/world/greta-thunberg-donate-covid-amazon-trnd/index.html?utm_medium=social&utm_content=2020-07-21T17%3A24%3A33&utm_source=twCNN&utm_term=link
5 - https://www.cnn.com/2020/07/21/business/winn-dixie-mask-policy-reversal/index.html?utm_term=link&utm_content=2020-07-21T16%3A46%3A05&utm_source=twCNN&utm_medium=social
6 - https://www.cnn.com/2020/07/21/politics/fact-check-trump-ad-biden-police-911/index.html?utm_term=link&utm_medium=social&utm_con

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1356, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 307, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.6/http/client.py", line 268, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/ssl.py", line 1012, in recv_into
    return self.read(nbytes, buffer)
  Fil

29 - https://www.cnn.com/2020/07/21/tech/linkedin-layoffs/index.html
30 - https://www.cnn.com/projects/ppp-business-loans/?utm_medium=social&utm_source=twCNN&utm_content=2020-07-21T11%3A02%3A05&utm_term=link
31 - https://www.cnn.com/2020/07/21/politics/joe-biden-caregiving-plan/index.html
32 - https://www.cnn.com/2020/07/15/health/n95-mask-inventor-coronavirus-sanjay-wellness-trnd/index.html?utm_medium=social&utm_content=2020-07-21T10%3A41%3A05&utm_term=link&utm_source=twCNN
33 - https://www.cnn.com/2020/07/21/health/racism-cognitive-function-study-wellness/index.html
34 - https://www.cnn.com/2020/07/20/us/alvin-ailey-dance-school-troy-powell-invs/index.html
35 - https://www.cnn.com/2020/07/21/politics/us-ukraine-navy/index.html
36 - https://www.cnn.com/2020/07/20/us/wall-of-moms-portland-protest-trnd/index.html
37 - https://www.cnn.com/2020/07/21/business/us-sanctions-china-companies-xinjiang-intl-hnk/index.html
38 - https://www.cnn.com/2020/07/21/us/san-francisco-giants-kneel-nationa

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/response.py", line 362, in _error_catcher
    yield
  File "/usr/local/lib/python3.6/dist-packages/urllib3/response.py", line 444, in read
    data = self._fp.read(amt)
  File "/usr/lib/python3.6/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/usr/lib/python3.6/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/ssl.py", line 1012, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.6/ssl.py", line 874, in read
    return self._sslobj.read(len, buffer)
  File "/usr/lib/python3.6/ssl.py", line 631, in read
    v = self._sslobj.read(len, buffer)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/l

219 - https://www.cnn.com/2020/07/14/us/alligator-kayak-trnd/index.html?utm_source=twCNN&utm_content=2020-07-19T10%3A31%3A06&utm_medium=social&utm_term=video
220 - https://www.cnn.com/2020/07/13/success/ford-bronco-2021-reveal/index.html?utm_term=video&utm_content=2020-07-19T10%3A16%3A04&utm_medium=social&utm_source=twCNN
221 - https://www.cnn.com/2020/07/16/world/solar-orbiter-first-closest-images-sun-campfires-scn-trnd/index.html?utm_content=2020-07-19T10%3A01%3A03&utm_medium=social&utm_source=twCNN&utm_term=link
222 - https://edition.cnn.com/travel/article/horse-island-ireland-sold-scli-intl/index.html?utm_term=video&utm_medium=social&utm_content=2020-07-19T09%3A31%3A06&utm_source=twCNN
223 - https://edition.cnn.com/travel/article/svalbard-norway-arctic-ice/index.html?utm_content=2020-07-19T09%3A16%3A03&utm_source=twCNN&utm_medium=social&utm_term=video
224 - https://www.cnn.com/2020/07/17/business/boeing-747-british-airways/index.html?utm_medium=social&utm_source=twCNN&utm_content=2

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/response.py", line 362, in _error_catcher
    yield
  File "/usr/local/lib/python3.6/dist-packages/urllib3/response.py", line 444, in read
    data = self._fp.read(amt)
  File "/usr/lib/python3.6/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/usr/lib/python3.6/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/ssl.py", line 1012, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.6/ssl.py", line 874, in read
    return self._sslobj.read(len, buffer)
  File "/usr/lib/python3.6/ssl.py", line 631, in read
    v = self._sslobj.read(len, buffer)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/l

733 - https://www.cnn.com/2020/07/14/tech/apple-slow-iphone-settlement-payouts/index.html?utm_medium=social&utm_content=2020-07-14T20%3A11%3A05&utm_source=twCNN&utm_term=link
734 - https://www.cnn.com/2020/07/14/economy/france-healthcare-workers-pay-intl/index.html?utm_term=link&utm_content=2020-07-14T20%3A01%3A04&utm_medium=social&utm_source=twCNN
735 - https://www.cnn.com/2020/07/14/business/best-buy-mask-requirement/index.html?utm_content=2020-07-14T19%3A51%3A02&utm_medium=social&utm_source=twCNN&utm_term=link
736 - https://www.cnn.com/2020/07/14/us/ghislaine-maxwell-pleads-not-guilty/index.html?utm_medium=social&utm_source=twCNN&utm_content=2020-07-14T19:41:25
737 - https://www.cnn.com/2020/07/14/politics/donald-trump-fauci-2020-campaign/index.html?utm_term=link&utm_content=2020-07-14T19%3A41%3A00&utm_medium=social&utm_source=twCNN
738 - https://www.cnn.com/2020/07/14/politics/immigration-harvard-visa-policy-online-only/index.html?utm_term=link&utm_content=2020-07-14T19%3A39%3A37&u

Prints list of end urls found

**Fetch URLs Backup file form github gist**

In [ ]:
news_urls = 'https://github.com/damithdev/public_files/blob/master/news_urls.csv?raw=true'

news_urls_df = pd.read_csv(news_urls)



**Fetch Article Data from the URL list**

Extract the article body from the html.
Store extracted data in article_data.csv.
Backup article_data.csv to google drive for future use

In [ ]:
print("Before fetching urls - "+ str(len(news_urls_df)))

articles = {}
errors = 0

itr = 0
for index, row in news_urls_df.iterrows():
  url = row['0']
  itr += 1
  
  try:
    r = requests.get(url)
    print(r.url)
    page = r.text
    soup = BeautifulSoup(page)
    title = soup.title 
    print(title)
    if title is not None:
      title = title.string
      content = ""
      flag = ""

      parsed_uri = urlparse(r.url)
      screen = '{uri.netloc}'.format(uri=parsed_uri)
      print(screen)
      if 'cnn' in screen:
          header = soup.find("h1", {"class": "pg-headline"})
          content = header.find_parent("div", {"class": "l-container"})
          flag = "CNN"
      elif 'bbc' in screen:
          flag = "BBC"
          content = soup.find(property="articleBody")
      else:
        raise Exception("Cannot Determine News Type")
      
      print(flag)
      title = title.string.split('- CNN')[0]
      print(title)
      if content is not None:articles[title]={'Title':title,'Content':content}
      print(str(len(articles)) + ":"+ title)
      print('=============================================================')
  except Exception as e:
    errors += 1
    print("exception")
    print(e)
    traceback.print_exc()

print("after fetching urls, articles based on unique titles (duplicate removal based on title) - "+ str(len(articles)))



csv_columns = ['Title','Content']
dict_data = []

for post in articles.values():
  dict_data.append(post)

# print(dict_data)

csv_file = "article_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
    
    print("Success")
    !cp article_data.csv "/content/gdrive/My Drive/colab_data"
except IOError:
    errors += 1
    print("I/O error")

print("Errors: " + str(articles))
print("Successful: "+ str(len(dict_data)))

Before fetching urls - 2678
https://www.cnn.com/2020/07/19/politics/coronavirus-stimulus-negotiations-white-house/index.html?utm_medium=social&utm_source=twCNN&utm_content=2020-07-20T12%3A45%3A02&utm_term=link
<title>Stimulus bill: White House chief of staff says stimulus negotiations will start 'in earnest' on Monday - CNNPolitics</title>
www.cnn.com
CNN
Stimulus bill: White House chief of staff says stimulus negotiations will start 'in earnest' on Monday 
1:Stimulus bill: White House chief of staff says stimulus negotiations will start 'in earnest' on Monday 
https://www.cnn.com/2020/07/20/asia/coronavirus-xinjiang-urumqi-intl-hnk/index.html?utm_term=link&utm_source=twCNN&utm_medium=social&utm_content=2020-07-20T12%3A30%3A09
<title>Xinjiang capital in lockdown over spike in coronavirus cases - CNN</title>
www.cnn.com
CNN
Xinjiang capital in lockdown over spike in coronavirus cases 
2:Xinjiang capital in lockdown over spike in coronavirus cases 
https://www.cnn.com/2020/07/19/entertai

Traceback (most recent call last):
  File "<ipython-input-19-984c3150d5c2>", line 28, in <module>
    content = header.find_parent("div", {"class": "l-container"})
AttributeError: 'NoneType' object has no attribute 'find_parent'


https://www.cnn.com/2020/07/12/us/atlanta-coronavirus-guitarist-skateboard-instagram-chavis-flagg-trnd/index.html?utm_term=video&utm_content=2020-07-20T07%3A00%3A12&utm_medium=social&utm_source=twCNN
<title>Atlanta artist Chavis Flagg won't be stopped from making music during the coronavirus pandemic - CNN</title>
www.cnn.com
CNN
Atlanta artist Chavis Flagg won't be stopped from making music during the coronavirus pandemic 
23:Atlanta artist Chavis Flagg won't be stopped from making music during the coronavirus pandemic 
https://edition.cnn.com/travel/article/svalbard-norway-arctic-ice/index.html?utm_content=2020-07-20T06%3A31%3A06&utm_source=twCNN&utm_medium=social&utm_term=video
<title data-rh="true">Svalbard, Norway: One woman, four men and weeks in the Arctic ice | CNN Travel</title>
edition.cnn.com
exception
'NoneType' object has no attribute 'find_parent'


Traceback (most recent call last):
  File "<ipython-input-19-984c3150d5c2>", line 28, in <module>
    content = header.find_parent("div", {"class": "l-container"})
AttributeError: 'NoneType' object has no attribute 'find_parent'


https://www.cnn.com/2020/07/16/world/metal-eating-bacteria-intl-scli-scn/index.html?utm_term=link&utm_content=2020-07-20T06%3A01%3A05&utm_source=twCNN&utm_medium=social
<title>Bacteria that eats metal accidentally discovered by scientists - CNN</title>
www.cnn.com
CNN
Bacteria that eats metal accidentally discovered by scientists 
24:Bacteria that eats metal accidentally discovered by scientists 
https://www.cnn.com/2020/07/15/health/n95-mask-inventor-coronavirus-sanjay-wellness-trnd/index.html?utm_term=link&utm_content=2020-07-20T05%3A05%3A03&utm_medium=social&utm_source=twCNN
<title>Peter Tsai, inventor of the N95 mask filter, is on a mission to help healthcare workers - CNN</title>
www.cnn.com
CNN
Peter Tsai, inventor of the N95 mask filter, is on a mission to help healthcare workers 
25:Peter Tsai, inventor of the N95 mask filter, is on a mission to help healthcare workers 
https://www.cnn.com/videos/business/2020/07/16/what-is-inflation-orig.cnn-business?utm_medium=social&utm_term

...

Traceback (most recent call last):
  File "<ipython-input-19-984c3150d5c2>", line 34, in <module>
    raise Exception("Cannot Determine News Type")
Exception: Cannot Determine News Type


https://www.bbc.com/news/world-africa-53150037?xtor=AL-72-%5Bpartner%5D-%5Bbbc.news.twitter%5D-%5Bheadline%5D-%5Bnews%5D-%5Bbizdev%5D-%5Bisapi%5D&at_custom1=%5Bpost+type%5D&at_medium=custom7&at_custom4=19D67194-B56D-11EA-A515-98F44744363C&at_custom2=twitter&at_custom3=%40BBCWorld&at_campaign=64
<title>Nigeria police rescue workers 'locked in rice factory' - BBC News</title>
www.bbc.com
BBC
Nigeria police rescue workers 'locked in rice factory' - BBC News
1541:Nigeria police rescue workers 'locked in rice factory' - BBC News
https://www.bbc.com/news/world-asia-53156264?xtor=AL-72-%5Bpartner%5D-%5Bbbc.news.twitter%5D-%5Bheadline%5D-%5Bnews%5D-%5Bbizdev%5D-%5Bisapi%5D&at_custom1=%5Bpost+type%5D&at_medium=custom7&at_custom4=FA089188-B56E-11EA-A515-98F44744363C&at_custom2=twitter&at_campaign=64&at_custom3=%40BBCWorld
<title>India expels dozens of Pakistan diplomatic staff - BBC News</title>
www.bbc.com
BBC
India expels dozens of Pakistan diplomatic staff - BBC News
1542:India expels dozens 

Traceback (most recent call last):
  File "<ipython-input-19-984c3150d5c2>", line 34, in <module>
    raise Exception("Cannot Determine News Type")
Exception: Cannot Determine News Type


https://www.bbc.com/news/world-europe-53113709?xtor=AL-72-%5Bpartner%5D-%5Bbbc.news.twitter%5D-%5Bheadline%5D-%5Bnews%5D-%5Bbizdev%5D-%5Bisapi%5D&at_custom4=8C2F1F46-B24F-11EA-99DB-78B94744363C&at_medium=custom7&at_custom1=%5Bpost+type%5D&at_custom3=%40BBCWorld&at_campaign=64&at_custom2=twitter
<title>Lough Keel: 'British father and son' recovered from Donegal lake - BBC News</title>
www.bbc.com
BBC
Lough Keel: 'British father and son' recovered from Donegal lake - BBC News
1632:Lough Keel: 'British father and son' recovered from Donegal lake - BBC News
https://www.bbc.com/news/world-asia-china-53113179?xtor=AL-72-%5Bpartner%5D-%5Bbbc.news.twitter%5D-%5Bheadline%5D-%5Bnews%5D-%5Bbizdev%5D-%5Bisapi%5D&at_custom2=twitter&at_custom3=%40BBCWorld&at_campaign=64&at_custom1=%5Bpost+type%5D&at_medium=custom7&at_custom4=E2DB0E3C-B249-11EA-99DB-78B94744363C
<title>Michael Kovrig and Michael Spavor: Canada renews calls for China to release charged pair - BBC News</title>
www.bbc.com
BBC
Michael K

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Prints the total number of urls before fetching

Prints the url and the extracted article's title

Prints the total number of articles after fetching and duplicate removal using article title

Displays number of failed attempts and successful attempts

### Answer (b)

Any pre-text explaining the code below...

**Fetch Article data csv from github (backed up)**

Reading csv into pandas dataframe

In [15]:
news_data_url = 'https://github.com/damithdev/public_files/blob/master/article_data.csv?raw=true'

df = pd.read_csv(news_data_url)

**Cleaning Data**

In [18]:
import nltk
nltk.download('punkt')
import re
import string
import collections


from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title', 'meta', 'Start', 'Post','Image']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

def removeOneTag(text, tag):
    return text[:text.find("<"+tag+">")] + text[text.find("</"+tag+">") + len(tag)+3:]

def removeBracketElement(text, tag):
    return text[:text.find("["+tag)] + text[text.find(tag+"]") + len(tag)+3:]

def apply_filter(dfcell):
  soup = BeautifulSoup(str(dfcell))
  data = soup.findAll(text=True)
  filtered = filter(visible, data)
  result =  ''.join(filtered)
  result = re.sub(r'[\t\r\n]', ' ', result)
  result = re.sub('Image copyright', ' ', result)
  result = re.sub('Getty Images', ' ', result)
  result = re.sub('Image caption', ' ', result)
  result = re.sub(r'\[if.+if\]', ' ', result)
  result = re.sub(r'Updated.+Messenger', ' ', result)
  result = re.sub(r' +', ' ', result)

  # print(result.strip())
  return result

def remove_characters_before_tokenization(sentence,
                                          keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence

def remove_characters_after_tokenization(tokens):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = list(filter(None, [pattern.sub('', token) for token in tokens]))
    return filtered_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Tokenize**

In [19]:
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences] 
    return word_tokens

**convert to lower case**

In [20]:
def tolower(corpus):
  return corpus.lower()


Apply filters to the dataframe to clean the data

In [21]:
df['filtered'] = df.Content.apply(apply_filter)

for each article corpus

*   prints the total number of words in the corpus body
*   removes unnessary characters before tokenization
*   expand the constractions of the cleaned corpus
*   tokenize the expanded corpus
*   remove characters after the tokenization
*   prints the number of unique words



In [27]:
tot_list = []
tot_words = 0
tot_u_words = 0
for corpus in df.filtered:
  print(corpus[0:100])
  corpus_tot_word = len(collections.Counter(corpus.split()))
  print("Total Words in corpus:"+str(corpus_tot_word))
  tot_words += corpus_tot_word

  cleaned_corpus = remove_characters_before_tokenization(tolower(corpus), keep_apostrophes=True) 

  # print(cleaned_corpus)

  expanded_corpus = expand_contractions(cleaned_corpus, CONTRACTION_MAP) 
  # print(expanded_corpus)      
  token_list = tokenize_text(expanded_corpus) 
  # print(token_list)

  filtered_list = list(filter(None,[remove_characters_after_tokenization(tokens) 
                                for tokens in token_list]))
  tot_list.append(filtered_list)
  # print(filtered_list)

  flat_list = [item for sublist in filtered_list for item in sublist]

  # print(flat_list)
  WORD_COUNTS = collections.Counter(flat_list)
  corpus_u_word = len(WORD_COUNTS)
  tot_u_word = corpus_u_word
  print("Unique Words in corpus:"+str(corpus_u_word))
  print("______________________")
  print()

print("__________________________________________")
print("==========================================")
print("TOTAL NUMBER OF WORDS DATASET: "+ str(tot_words))
print("TOTAL NUMBER OF UNIQUE WORDS DATASET: " + str(tot_u_words))
print("__________________________________________")

Unique Words in corpus:382
______________________

 The lawsuit seeks a restraining order to stop federal agents detaining protesters in Portland The a
Total Words in corpus:469
Unique Words in corpus:400
______________________

 India currently has the third-highest number of confirmed cases of coronavirus The number of new ca
Total Words in corpus:424
Unique Words in corpus:393
______________________

 King Juan Carlos is under investigation in Spain and Switzerland A former lover's taped conversatio
Total Words in corpus:761
Unique Words in corpus:680
______________________

 Women around the world are much more likely than men to wear a face mask, evidence shows After much
Total Words in corpus:647
Unique Words in corpus:573
______________________

 Amid the global debate about race relations, colonialism and slavery, some of the Europeans and Ame
Total Words in corpus:790
Unique Words in corpus:696
______________________

 The Shady Maples Smorgasbord - specialising in Amish food 